# SCC.413 Applied Data Mining
# Week 18
# Classification 2 (using custom Document)

## Contents
* [Introduction](#intro)
* [Preamble](#preamble)
* [Document object](#doc)
    - [Dataset](#data)
    - [Document processor](#processor)
* [Feature Union](#union)
* [Classifying other labels](#other)
* [Exercise](#ex)

<a name="intro"></a>
## Introduction

In the the first classification notebook scikit-learn was used to classify Twitter users as male or female. Vectorization was done via sklearn's CountVectorizer. At the end of the lab exercise, the preprocessing and tokenisation used by CountVectorizer were customised to give greater control of how the text was processed and features extracted. This actually allows for quite a number of features to be implemented, e.g.:

- POS tags, by pos-tagging during tokenisation, and returning the pos-tags instead of words.
- Function words, by setting the vocabulary of CountVectorizer to a function word list.
- Hashtags, mentions and/or emojis, by only returning these in the token list.
- Characters/graphemes, rather than codepoints, by "tokenising" with "\X".

But we are limited to counting things. Another issue is that the processing will be done multiple times each time the pipeline is ran (e.g. for gridsearch).

In this lab you will learn how to process texts in advance, saving the token lists and frequency lists produced, and allowing for other features to be extracted.

<a name="preamble"></a>
## Preamble

First, we do all the imports needed in one go. You may import other packages as needed.

In [ ]:
!pip install ftfy

In [ ]:
import ftfy
import nltk
import json

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Binarizer, StandardScaler

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re

from collections import Counter
from os import listdir, makedirs
from os.path import isfile, join, splitext, split

As in previous notebooks, you should upload all of the provided files to a Google Drive folder, you can then access these files from your Python code. See also the files tab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

We save the folder we are working from as a variable for easy access. You may need to edit the path to match your own.

In [ ]:
working_folder = '/content/gdrive/MyDrive/413/wk19/'

The below code adds the working folder to the system path, so you can import Python files from this folder.

In [ ]:
import sys
sys.path.append(working_folder)

A couple of functions for showing classifier results (from first classification lab):

In [ ]:
def print_cv_scores_summary(name, scores):
    print("{}: mean = {:.2f}%, sd = {:.2f}%, min = {:.2f}, max = {:.2f}".format(name, scores.mean()*100, scores.std()*100, scores.min()*100, scores.max()*100))
    
def confusion_matrix_heatmap(cm, index):
    cmdf = pd.DataFrame(cm, index = index, columns=index)
    dims = (5, 5)
    fig, ax = plt.subplots(figsize=dims)
    sns.heatmap(cmdf, annot=True, cmap="coolwarm", center=0)
    ax.set_ylabel('Actual')    
    ax.set_xlabel('Predicted')

<a name="doc"></a>
## Document object

Next we define our document object, along with preprocessing tokenisation methods. These are taken (and slightly extended) from the Feature Extraction lab. The document will represent an instance in our classifier, e.g. it could be a collection of a user's Tweets, a single tweet, a longer article, etc.

In [ ]:
hashtag_re = re.compile(r"#\w+")
mention_re = re.compile(r"@\w+")
url_re = re.compile(r"(?:https?://)?(?:[-\w]+\.)+[a-zA-Z]{2,9}[-\w/#~:;.?+=&%@~]*")

def preprocess(text):
    p_text = hashtag_re.sub("[hashtag]",text)
    p_text = mention_re.sub("[mention]",p_text)
    p_text = url_re.sub("[url]",p_text)
    p_text = ftfy.fix_text(p_text)
    return p_text

tokenise_re = re.compile(r"(\[[^\]]+\]|[-'\w]+|[^\s\w\[']+)") #([]|words|other non-space)
def tokenise(text):
    return tokenise_re.findall(text)

class Document:
    def __init__(self, meta={}):
        self.meta = meta
        self.tokens_fql = Counter() #empty Counter, ready to be added to with Counter.update.
        self.ht_fql = Counter()
        self.num_tokens = 0
        
    def extract_features_from_text(self, text):
        hts = hashtag_re.findall(text)
        self.ht_fql.update([ht.lower() for ht in hts])
        p_text = preprocess(text)
        tokens = tokenise(p_text)
        lower_tokens = [t.lower() for t in tokens]
        self.num_tokens += len(lower_tokens)
        self.tokens_fql.update(lower_tokens) #updating Counter counts items in list, adding to existing Counter items.
        
    def extract_features_from_texts(self, texts): #texts should be iterable text lines, e.g. read in from file.
        for text in texts:
            extract_features_from_text(text)
            
    def average_token_length(self):
        sum_lengths = 0
        for key, value in self.tokens_fql.items():
            sum_lengths += len(key) * value
        return sum_lengths / self.num_tokens

<a name="data"></a>
### Dataset

We use the Twitter celebs again (this time with extra users from around the world), but from JSON files with further metadata available. The method below reads in the json file, extracts the metadata for the user, creates a new Document representing the user, and adds all of the Tweets (extracting features).

In [ ]:
def import_celebs_json(folder):
    jsonfiles = [join(folder, f) for f in listdir(folder) if isfile(join(folder, f)) and f.endswith(".json")]
    for jf in jsonfiles:
        with open(jf) as f:
            data = json.load(f)
            handle = data['handle']
            gender = data['gender']
            age_range = data['age_range']
            english = data['english']
        print("Processing " + handle)
        doc = Document(meta={'handle': handle, 'gender': gender, 'age_range': age_range, 'english': english}) #include metadata
        for tweet in data['tweets']:
            doc.extract_features_from_text(tweet['full_text'])
        yield doc

Build the corpus by processing the folder of celebs. This will take a little while, but it should only need doing once (unless you change the Document class). We are using just the gb users here, but you can use one of the bigger datasets if you wish.

In [ ]:
corpus = []
corpus.extend(import_celebs_json(working_folder + "celebs-gb-json"))

We use the gender metadata from the Documents as our label/class.

In [ ]:
y = [d.meta['gender'] for d in corpus]
X = corpus

We can also perform the usual train/test split.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state = 0, stratify=y)
print(len(X_train), len(X_test))
print(len(y_train), len(y_test))

<a name="processor"></a>
### Document processor

Now we have our train/test instances in our custom class to hold the features, we need to build a custom `Transformer` which takes in one dataset and returns a new dataset. Here we need to take in a list of `Document` objects and transform it into a set of features. We build a simple class for this, which overrides the transform method. The intention is for a list of `Document` objects to be passed into the transformer, and a parameter-defined (callable) method is used to extract the featuress.

In [ ]:
class DocumentProcessor(BaseEstimator, TransformerMixin):
    def __init__(self, process_method):
        self.process_method = process_method
    
    def fit(self, X, y=None): #no fitting necessary, although could use this to build a vocabulary for all documents, and then limit to set (e.g. top 1000).
        return self

    def transform(self, documents):
        for document in documents:
            yield self.process_method(document)

Below are some example methods for returning the token frequency list, hashtag frequency list, or some text statistics from the `Document`. These can be edited and added to as needed.

In [ ]:
def get_tokens_fql(document):
    return document.tokens_fql

def get_ht_fql(document):
    return document.ht_fql

def get_text_stats(document):
    ttr = len(document.tokens_fql) / document.num_tokens
    return {'avg_token_length': document.average_token_length(), 'ttr': ttr }

def read_list(file):
    with open(file) as f:
        items = []
        lines = f.readlines()
        for line in lines:
            items.append(line.strip())
    return items

fws = read_list(working_folder + "functionwords.txt")

def get_fws_fql(document):
    fws_fql = Counter({t: document.tokens_fql[t] for t in fws}) #dict comprehension, t: fql[t] is token: freq.
    return +fws_fql

We build a pipeline with the `DocumentProcessor` as the first step, extract hashtag frequencies as features. The output from the new `DocumentProcessor` is then passed to a [`DictVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which transforms the features into a vector (scipy.sparse matrix), which can be used with other sklearn steps. To demontrate, we pass to a logistic regression classifier.

In [ ]:
model = Pipeline([
    ('processor', DocumentProcessor(process_method = get_ht_fql)),
    ('vectorizer', DictVectorizer()),
    ('standardize', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(solver='liblinear', random_state=0)),
])

In [ ]:
cv_scores = cross_validate(model, X, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0), return_train_score=False, scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
print_cv_scores_summary("Accuracy", cv_scores['test_accuracy'])
print_cv_scores_summary("Precision", cv_scores['test_precision_weighted'])
print_cv_scores_summary("Recall", cv_scores['test_recall_weighted'])
print_cv_scores_summary("F1", cv_scores['test_f1_weighted'])

<a name="union"></a>
## Feature Union

All the pipelines we have used so far have been simple linear sequences of steps. With [`FeatureUnion`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion) we can have complex branched sequences, where different features, or different parts of the text are processed separately. The results are then concatenated together into a composite vector. We could, for example, read in a user's bio separately from the rest of their text, we could even utilise metadata (such as usernames to predict gender), or other data such as profile images.

Below we utilise different featuresets available from the Document instances and concatenate them together simply.

In [ ]:
model = Pipeline([
    ('text_union', FeatureUnion(
        transformer_list = [
            ('ht_features', Pipeline([
                ('ht_processor', DocumentProcessor(process_method = get_ht_fql)),
                ('ht_vectorizer', DictVectorizer()),
            ])),
            ('fws_features', Pipeline([
                ('fws_processor', DocumentProcessor(process_method = get_fws_fql)),
                ('fws_vectorizer', DictVectorizer()),
            ])),
            ('stats_features', Pipeline([
                ('stats_processor', DocumentProcessor(process_method = get_text_stats)),
                ('stats_vectorizer', DictVectorizer()),
            ])),
        ],
    )),
    ('standardize', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(solver='liblinear', random_state=0)),
])

In [ ]:
cv_scores = cross_validate(model, X, y, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=0), return_train_score=False, scoring=['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'])
print_cv_scores_summary("Accuracy", cv_scores['test_accuracy'])
print_cv_scores_summary("Precision", cv_scores['test_precision_weighted'])
print_cv_scores_summary("Recall", cv_scores['test_recall_weighted'])
print_cv_scores_summary("F1", cv_scores['test_f1_weighted'])

<a name="other"></a>
## Classifying other labels

We have been working on binary classification of gender. Other user metadata can be predicted also, with the age_range and English variety being present (if you use the larger datasets provided).

Looking at age, there are 5 age ranges present, plus some marked as "unknown". We need to remove the "unknowns" as predicting as 'unknown' does not make sense. You could predict these unknowns with your trained classifier at the end.

In [ ]:
set([d.meta['age_range'] for d in X])

In [ ]:
age_X = [d for d in X if d.meta['age_range'] != 'unknown']

In [ ]:
len(age_X)

We need to extract the new labels also.

In [ ]:
age_y = [d.meta['age_range'] for d in age_X]

We do a train-test split as before.

In [ ]:
age_X_train, age_X_test, age_y_train, age_y_test = train_test_split(age_X, age_y, test_size=0.2, random_state = 0, stratify=age_y)

We build a pipeline with feature union similar to earlier, except we are now also using grid search, including if to include function words or all words.

In [ ]:
model = Pipeline([
    ('union', FeatureUnion(
        transformer_list = [
            ('ht', Pipeline([
                ('processor', DocumentProcessor(process_method = get_ht_fql)),
                ('vectorizer', DictVectorizer()),
            ])),
            ('word', Pipeline([
                ('processor', DocumentProcessor(process_method = None)), # to be set by grid search.
                ('vectorizer', DictVectorizer()),
            ])),
            ('stats', Pipeline([
                ('processor', DocumentProcessor(process_method = get_text_stats)),
                ('vectorizer', DictVectorizer()),
            ])),
        ],
    )),
    ('selector', SelectKBest(score_func = chi2)),
    ('standardize', StandardScaler(with_mean=False)),
    ('clf', LogisticRegression(solver='liblinear', random_state=0, multi_class='ovr')),
])

param_grid={
    'union__word__processor__process_method': [get_fws_fql, get_tokens_fql],
    'selector__k': [50, 100, 150, 500],
}



search = GridSearchCV(model, cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0), 
                      return_train_score = False, 
                      scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted'],
                      refit = 'f1_weighted',
                      param_grid = param_grid
                     )

search.fit(age_X_train, age_y_train)


In [ ]:
pd.DataFrame(search.cv_results_)

In [ ]:
search.best_params_

In [ ]:
predictions = search.predict(age_X_test)

print("Accuracy: ", accuracy_score(age_y_test, predictions))
print(classification_report(age_y_test, predictions))
print(confusion_matrix(age_y_test, predictions))

confusion_matrix_heatmap(confusion_matrix(age_y_test,predictions), search.classes_)

You can save your model for later use. This is straightforward with "pickling", which is serialising Python objects to files. There is a nice intro to pickling here: https://medium.com/better-programming/dont-fear-the-pickle-using-pickle-dump-and-pickle-load-5212f23dbbce

You can easily save a built model, e.g. the best model from a grid search, and load it for later use (e.g. making predictions).

In [ ]:
import pickle
with open('age_model.pkl', 'wb') as f:
    pickle.dump(search, f)

And load an existing model with:

In [ ]:
with open('age_model.pkl', 'rb') as f:
    age_model = pickle.load(f)

In [ ]:
age_model.best_params_

In [ ]:
predictions = age_model.predict(age_X_test)

print("Accuracy: ", accuracy_score(age_y_test, predictions))
print(classification_report(age_y_test, predictions))
print(confusion_matrix(age_y_test, predictions))

confusion_matrix_heatmap(confusion_matrix(age_y_test,predictions), search.classes_)

<a name="ex"></a>
## Exercise

Try adding and improving the features used for age classification, and evaluate your results with different setups.


### Advanced tasks:

- You could try predicting the precise age with a regression model (the age is in the json metadata). See: https://dl.acm.org/citation.cfm?id=2107651.
- Or you could try predicting age and gender together, either mutli-class pairs, or through a classification tree, e.g. predict gender first, then narrowing age ranges.